# This represents a notebook for testing GAN architectures

In [4]:
print('Test')

Password:··········
Cloning into 'master_thesis'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
[Errno 2] No such file or directory: 'master_thesis'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
